In [9]:
import os
import pandas as pd

dir0 = 'data0/'
dir1 = 'data1/'

def rename_columns(dir0, dir1):
    for file in os.listdir(dir0):
        if file.endswith('.csv'):
            df = pd.read_csv(dir0 + file)
            if file == 'policies.csv':
                df.to_json('json_files/' + file.replace('.csv', '.json'), orient='records')
                continue

            df.columns = df.columns.str.lower()
            df.rename(columns={'geo': 'country', 'time_period': 'year', 'obs_value': 'value'}, inplace=True)
            df.drop(columns=['dataflow', 'last update', 'freq', 'obs_flag', 'conf_status'], inplace=True, errors='ignore')

            df = df[~df['country'].isin(['Türkiye', 'Iceland', 'Switzerland', 'Norway', 'Montenegro', 'North Macedonia', 'Serbia', 'Albania', 'Bosnia and Herzegovina', 'Kosovo*', 'Liechtenstein', 'Moldova', 'United Kingdom'])]

            # Print every country that is NOT in the EU
            countries_eu = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
            countries_not_eu = df[~df['country'].isin(countries_eu)]['country'].unique()
            if len(countries_not_eu) > 0:
                print(countries_not_eu)

            
            if 'unit' in df.columns:
                pivot_index = ['country', 'unit']
                main_columns = ['indicator', 'country', 'unit']
            else:
                pivot_index = ['country']
                main_columns = ['indicator', 'country']
                
            df_wide = df.pivot_table(index=pivot_index, columns='year', values='value', aggfunc='first').reset_index()

            df_wide['indicator'] = file.replace('.csv', '')
            
            year_columns = sorted([col for col in df_wide.columns if col not in main_columns])
            df_wide = df_wide[main_columns + year_columns]
            
            df_wide.to_csv(dir1 + file, index=False)
            df_wide.to_json('json_files/' + file.replace('.csv', '.json'), orient='records')

rename_columns(dir0, dir1)

['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)'
 'European Union - 28 countries (2013-2020)']
['European Commission' 'Czechia' 'European Investment Bank (EIB)'
 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'Euro area – 20 countries (from 2023)'
 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union (aggregate changing according to the context)']
['Czechia' 'European Union - 27 countries (from 2020)']
['Czechia' 'European Union - 27 countries (from 2020)'
 